What I need:

Which information to put into db?

Data engineering to make it better:

-labels?
    - have to create
    -types of rsos:
        
        
        -finance

        - investing
        - private equity
        

        -marketing

        -tech

        -science?

            -bio
            -chem
            -physics
            -engineering?
        -sport

        -humanities

        -art?



todo:
-db
-chatbot
-data engineering (labelling, other stuff?)

In [1]:
import json
from typing import Dict, Any

In [2]:
def transform_rso_data(rso_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Transform RSO data for Pinecone database.
    
    Args:
        rso_data (Dict[str, Any]): Original RSO data dictionary
        
    Returns:
        Dict[str, Any]: Transformed data ready for Pinecone
    """
    # Extract high confidence AI categories
    high_confidence_categories = [
        cat["name"] 
        for cat in rso_data["ai_categories"] 
        if cat["confidence"] >= 85
    ]
    
    # Combine with original categories, remove duplicates
    all_categories = list(set(rso_data["categories"] + high_confidence_categories))
    
    # Use full_description if available, otherwise fall back to description_preview
    description = (
        rso_data["full_description"]
        if rso_data.get("full_description")
        else rso_data.get("description_preview", "")
    )
    
    # Create transformed dictionary with only desired fields
    transformed_data = {
        "name": rso_data["name"],
        "full_url": rso_data["full_url"],
        "description": description,  # Renamed field to be more generic
        "categories": all_categories,
        "contact": {"email": rso_data["contact"]["email"]},
        "additional_info": rso_data["additional_info"],
        "social_media": rso_data["social_media"]
    }
    
    return transformed_data

In [3]:
def process_file(input_path: str, output_path: str):
    """
    Process an entire JSON file containing RSO data.
    
    Args:
        input_path (str): Path to input JSON file
        output_path (str): Path to write transformed JSON file
    """
    try:
        # Read input file
        with open(input_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Handle both single object and list of objects
        if isinstance(data, list):
            transformed_data = [transform_rso_data(item) for item in data]
        else:
            transformed_data = transform_rso_data(data)
        
        # Write transformed data to output file
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(transformed_data, f, indent=2, ensure_ascii=False)
            
        print(f"Successfully transformed data and saved to {output_path}")
        
    except FileNotFoundError:
        print(f"Error: Could not find input file {input_path}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON in input file {input_path}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {str(e)}")

In [ ]:


input_file = "rso_data.json"
output_file = "transformed_rso_data.json"
process_file(input_file, output_file)